In [ ]:
!pip install independent_vector_analysis
!pip install librosa

In [ ]:
import numpy as np
from independent_vector_analysis import iva_g, consistent_iva
import librosa
import soundfile as sf
import IPython.display as ipd
import librosa.display
import librosa.effects
import librosa.feature

from scipy.signal import iirnotch, butter, lfilter


In [ ]:
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    filtered_data = lfilter(b, a, data)
    return filtered_data

In [ ]:
#loading audio
sr = 44100
audio, sample_rate = librosa.load("/content/Clara Berry And Wooldog - Stella.stem.mp4", sr=sr, mono=False)
a = librosa.effects.preemphasis(audio, coef=0.97)
s = librosa.core.stft(a)


N=2 #Number of sources are 2 for vocal and instrument
K=2 #Number of mixtures will be 2 for a stereo channel
T = len(s[0]) #No. of Samples

S = np.zeros((N, T, K))  # Initialize X with zeros
for n in range(N):  # For each source
  S[:, :, n] = s.T[:, n]

A = np.random.randn(N,N,K)
X = np.einsum('MNK, NTK -> MTK', A, S)

W, cost, Sigma_n, isi = iva_g(X, A=A)
#print(W)

<ipython-input-33-61a03998599d>:3: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sample_rate = librosa.load("/content/Clara Berry And Wooldog - Stella.stem.mp4", sr=sr, mono=False)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [ ]:
separated_sources = np.matmul(W, s)
i = librosa.core.istft(seperated_sources)

stereo_data = i[0,:,:]
stereo_data = np.transpose(stereo_data)
file_name = f'source_1.wav'
sf.write(file_name, stereo_data, sample_rate)
stereo_data = i[1,:,:]
stereo_data = np.transpose(stereo_data)
file_name = f'source_2.wav'
sf.write(file_name, stereo_data, sample_rate)

(2, 2, 8624128)


In [ ]:
from IPython.display import Audio
Audio("/content/source_vocal.wav")